# <center> Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком


## <center> Описание используемых данных

Данные взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. 

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных описана количественная мера активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

В рамках задачи следует определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, интересно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/gene_high_throughput_sequencing.csv')

In [3]:
data.shape

(72, 15750)

In [4]:
data.groupby('Diagnosis').agg({'Patient_id':'count'})

,Patient_id
Diagnosis,
cancer,23
early neoplasia,25
normal,24


## Применение t-критерия Стьюдента

Применим критерий стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках.

Для каждого гена критерий проверим дважды, normal - early_neoplasia & early_neoplasia - cancer

Укажем количество статистически значимых отличий, которые нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [5]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 8.7+ MB


In [7]:
indexes = data.select_dtypes(include = 'float64').columns

In [8]:
data_norm = data[data['Diagnosis'] == 'normal'].select_dtypes(include = 'float64')
data_neo = data[data['Diagnosis'] == 'early neoplasia'].select_dtypes(include = 'float64')
data_canc = data[data['Diagnosis'] == 'cancer'].select_dtypes(include = 'float64')

In [9]:
norm_neo = pd.DataFrame(columns = ['norm_neo_p_val'], index = indexes )
neo_canc = pd.DataFrame(columns = ['neo_can_p_val'], index = indexes )

In [10]:
from scipy.stats import ttest_ind

In [11]:
for gen in indexes:
    p_val = ttest_ind(data_norm[gen],data_neo[gen], equal_var = False).pvalue
    norm_neo.loc[gen,:] = p_val
    

In [12]:
for gen in indexes:
    p_val = ttest_ind(data_neo[gen],data_canc[gen], equal_var = False).pvalue
    neo_canc.loc[gen,:] = p_val

In [13]:
answer1 = sum(norm_neo.iloc[:,0] < 0.05)
answer2 = sum(neo_canc.iloc[:,0] < 0.05) 

In [14]:
print(f'Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = {answer1}')

Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = 1575


In [15]:
print(f'Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = {answer2}')

Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = 3490


## <center> Поправка на множественную проверку гипотез и практическую значимость отличий

* При множественном сравнении гипотез, появляется так называемый "эффект множественных сравнений". Во время проверки статистических гипотез при отвержении основной гипотезы ($H_0$) возможна ошибка (ложное отклонение гипотезы, ошибка первого рода). Вероятность такого события ограничивается неким малым предварительно выбранным значением — уровнем значимости ${\displaystyle \alpha }$  (обычно ${\displaystyle \alpha =0{,}05}$). Тогда при построении ${\displaystyle m}$ выводов верхняя оценка вероятности того, что хотя бы один из них будет неверным, равна ${\displaystyle 1-(1-\alpha )^{m}}$, что достаточно велико уже при небольших ${\displaystyle m}$ (например, при ${\displaystyle m=5}$, ${\displaystyle \alpha =0{,}05}$ она равна ${\displaystyle \approx 22{,}6\%})$ . Для устранения этого эффекта было разработано несколько подходов.
    
* Кроме этого надо учитывать практическую значимость, в экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:
    
    $${\displaystyle F_c (C,T) = {\frac{T}{C}}{,}} \: {T > C}$$ 
    
    $${\displaystyle F_c (C,T) = {\frac{C}{T}}{,}} \: {C > T}$$

    где C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Поправка методом Холма

In [16]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [17]:
norm_neo['p_holm'] = multipletests(norm_neo['norm_neo_p_val'], alpha = 0.025, method = 'holm')[1]
neo_canc['p_holm'] = multipletests(neo_canc['neo_can_p_val'], alpha = 0.025, method = 'holm')[1]

In [18]:
data_norm_np = data_norm.values
data_neo_np = data_neo.values
data_canc_np = data_canc.values

In [19]:
def F(control, treatment):
    F_values = []
    for i in range(len(indexes)):
        mean_control = abs(control[:,i].mean())
        mean_treatment = abs(treatment[:,i].mean())
        if mean_treatment > mean_control:
            F = abs(mean_treatment / mean_control)
        else:
            F = abs(-mean_control / mean_treatment)   
        F_values.append(F)
    return F_values
        

In [20]:
norm_neo['F'] = F(data_norm_np, data_neo_np)
neo_canc['F'] = F(data_neo_np, data_canc_np)

In [21]:
answer3 = sum(((norm_neo['F'] > 1.5) & (norm_neo['p_holm'] < 0.025)))

In [22]:
answer4 = sum(((neo_canc['F'] > 1.5) & (neo_canc['p_holm'] < 0.025)))

In [23]:
print(f'Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = {answer3}')

Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = 2


In [24]:
print(f'Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = {answer4}')

Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = 77


### Поправка методом Бенджамини-Хохберга

In [25]:
norm_neo['p_bh'] = multipletests(norm_neo['norm_neo_p_val'], alpha = 0.025, method = 'fdr_bh')[1]
neo_canc['p_bh'] = multipletests(neo_canc['neo_can_p_val'], alpha = 0.025, method = 'fdr_bh')[1]

In [26]:
answer5 = sum(((norm_neo['F'] > 1.5) & (norm_neo['p_bh'] < 0.025)))
answer6 = sum(((neo_canc['F'] > 1.5) & (neo_canc['p_bh'] < 0.025)))

In [27]:
print(f'Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = {answer5}')

Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = 4


In [28]:
print(f'Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = {answer6}')

Количество статистически значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = 524


## Итоги
* t - критерий стюьюдента определил **1575** значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" и **3490** значимых отличий в генах у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии. 
* Необходимо было сделать поправку на практическюу значимость отличий и множественную проверку гипотез, использовалось два подхода:
    - Поправка методом Холма:
        * Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = **2** и у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = **77**
    - Поправка методом Бенджамини-Хохберга
        * Количество статистически значимых отличий в генах у групп "Не болел раком груди" и "Рак диагностирован на ранней стадии" = **4** и у групп "Сильно выражены симптомы Рака" и "Рак диагностирован на ранней стадии" = **524**
* Метод Холма — это нисходящая процедура множественной проверки гипотез и он позволяет не совершить ни одной ошибки первого рода, при этом допуская большое количество ошибок второго рода, в то же время метод Бенджамини-Хохберг подразумевает восходящую процедуру множественной проверки гипотез, соответственно он будет совершать ошибки первого рода, но в то же время ошибок второго рода будет значительно меньше чем при использовании метода Холма, соблюдается некий баланс между ошибками второго и первого рода, поэтому значимых отличий он определил больше (524) по сравнению с методом Холма (77) 
